In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import yfinance as yf
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

# Initialize the Fake News Detection model
FAKE_NEWS_MODEL = "jy46604790/Fake-News-Bert-Detect"
fake_news_clf = pipeline("text-classification", model=FAKE_NEWS_MODEL, tokenizer=FAKE_NEWS_MODEL)

# Initialize the Sentiment Analysis model (FinBERT)
SENTIMENT_MODEL = "ProsusAI/finbert"
sentiment_tokenizer = AutoTokenizer.from_pretrained(SENTIMENT_MODEL)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(SENTIMENT_MODEL)
sentiment_clf = pipeline("text-classification", model=sentiment_model, tokenizer=sentiment_tokenizer)

# for validating news using the Fake News Detection model
def check_fake_news(news_text):
    result = fake_news_clf(news_text)
    label = result[0]['label']  # LABEL_1 is Real, LABEL_0 is Fake
    return label == 'LABEL_1'

# for analyzing sentiment using the Sentiment Analysis model
def analyze_sentiment(news_text):
    sentiment_result = sentiment_clf(news_text)
    return sentiment_result[0]['label']  # Returns the sentiment label

# for fetching stock data and predict prices
def predict_stock_prices(ticker, sentiment):
    # Fetch historical data
    stock_data = yf.download(ticker, period="6mo", interval="1d")
    if stock_data.empty:
        return None, None

    # Prepare data for regression
    stock_data['Days'] = np.arange(len(stock_data))
    X = stock_data[['Days']]
    y = stock_data['Close']

    # Apply linear regression
    model = LinearRegression()
    model.fit(X, y)
    future_days = np.array([len(stock_data) + i for i in range(1, 6)]).reshape(-1, 1)  # Predict next 5 days
    predicted_prices = model.predict(future_days)

    # Adjust predictions based on sentiment
    adjustment_factor = 0.05  # 5% adjustment for sentiment
    if sentiment == "POSITIVE":
        predicted_prices *= (1 + adjustment_factor)
    elif sentiment == "NEGATIVE":
        predicted_prices *= (1 - adjustment_factor)

    # Return original and adjusted predictions
    return future_days.flatten(), predicted_prices


def main():
    while True:
        # Input news text
        news_text = input("Enter a news article: ")

        # Check if the news is fake
        is_real = check_fake_news(news_text)
        if not is_real:
            print("The news you entered is fake. Please try again.")
            continue  # Ask for input again if fake

        # Analyze sentiment of the real news
        sentiment = analyze_sentiment(news_text)
        print(f"The news is real and its sentiment is: {sentiment}")

        # Input stock ticker
        ticker = input("Enter a stock ticker (e.g., MSFT, AAPL): ").upper()

        # Predict stock prices
        future_days, predicted_prices = predict_stock_prices(ticker, sentiment)
        if future_days is None:
            print(f"Could not fetch data for ticker {ticker}. Please try another.")
            continue

        # Display results
        print(f"Predicted prices for {ticker} over the next 5 days based on sentiment '{sentiment}':")
        for day, price in zip(future_days, predicted_prices):
            print(f"Day {int(day)}: ${price}")

        break  # exit after successful prediction

if __name__ == "__main__":
    main()


Enter a news article: After Trump won United States elections, the stock market will grow up for a short period
The news you entered is fake. Please try again.
Enter a news article: After Trump won United States elections, the stock market grew up in USA
The news you entered is fake. Please try again.
Enter a news article: If Trump wins the US elecitons, the US market will grow
The news you entered is fake. Please try again.
Enter a news article: Trump promised he will end the Ukraine war if  he would win the presidential elections in US
The news you entered is fake. Please try again.
Enter a news article: The stocks market in US prices grew in last month
The news is real and its sentiment is: positive
Enter a stock ticker (e.g., MSFT, AAPL): AAPL


[*********************100%***********************]  1 of 1 completed

Predicted prices for AAPL over the next 5 days based on sentiment 'positive':
Day 129: $[235.20358104]
Day 130: $[235.43280794]
Day 131: $[235.66203484]
Day 132: $[235.89126174]
Day 133: $[236.12048865]
